In [15]:
from operator import itemgetter
import random
import matplotlib.pyplot as plt
import numpy as np
import queue
import pandas as pd
import requests as r
import pandas as pd
import random,time
import threading
from datetime import datetime, date
from io import StringIO
import plotly.express as pe
import plotly.graph_objects as go
from multiprocessing import Pool
from multiprocessing import Process ,  Queue
from itertools import repeat
from TwoMA import TwoMA_Fitness_Process , TwoMA_decode , TwoMA_validate , TwoMA
# from TwoMA_with_handling_fee import TwoMA_Fitness_Process , TwoMA_decode , TwoMA_validate , TwoMA
from Bollinger_Band import BollingerBand_Fitness_Process , BollingerBand_validate , BollingerBand_decode , BollingerBand
# from TwoMA import TwoMA , Fitness_Process

In [16]:

#初始種群
def init_population(pop_size,dna_length):
    population = []
    np.random.seed(1108)   #隨機種子
    for num in range(pop_size):
      chromosome = ''
      for len in range(dna_length):
        chromosome = chromosome+str(np.random.randint(0,2))
      
      population.append(chromosome)

    return population







#計算適應度
def evaluate_fitness(pop , period ,  training_data , init_money):
    fitness = []
    

    
    # processes數量
    CPU_Core_num = 6
    pool = Pool(processes = CPU_Core_num)
    # 把全部資料丟進process跑 checksize = population / proccess數量
    cal_result = pool.starmap(TwoMA_Fitness_Process , zip(pop, repeat(training_data) , repeat(period) , repeat (init_money)) , chunksize = int(len(pop) / CPU_Core_num) )
    # cal_result = pool.starmap(BollingerBand_Fitness_Process , zip(pop, repeat(training_data) , repeat(period) , repeat (init_money)) , chunksize = int(len(pop) / CPU_Core_num) )
    


    for r in range(len(pop)):
        fitness.append([pop[r] , cal_result[r]])
    

    return max(cal_result) , sum(cal_result)/len(pop) , fitness
    


#選擇 & 交配
def cross(pop,POP_SIZE,DNA_LEN,cross_rate):
    new_pop = []
    while len(new_pop) != POP_SIZE:
        ran_F = random.choices(pop, k=2)
        ran_M = random.choices(pop, k=2)
        father = max(ran_F, key = itemgetter(1))[0]
        mother = max(ran_M, key = itemgetter(1))[0]

        if np.random.rand() < cross_rate:
            cross_point = np.random.randint(low=0, high = DNA_LEN)
            cut_F = father[cross_point:]
            cut_M = mother[cross_point:]
            father = father[:cross_point]+cut_M
            mother = mother[:cross_point]+cut_F

        new_pop.extend([father,mother])

    return new_pop


#變異
def mutation(pop,DNA_LEN,mutation_rate):
    
    for m in range(len(pop)):
        if np.random.rand() < mutation_rate:
          mutation_gene  = np.random.randint(DNA_LEN)
          mu_chromosome = list(pop[m])
          if mu_chromosome[mutation_gene] == '1':
            mu_chromosome[mutation_gene] = '0'
          else:
            mu_chromosome[mutation_gene] = '1'
          
          pop[m] = ''.join(mu_chromosome)

    return pop


#繪製種群狀態
def draw_fitness(run,best_pop,avg_pop):
    plt.title("Best & Avg fitness")  #圖表標題
    plt.xlabel("Generation")  #圖表x軸標題
    plt.ylabel("fitness")  #圖表y軸標題
    plt.plot(run, best_pop, color='r', label='Best fitness', linestyle='-', marker='*')   #定義畫圖的顏色、圖例、線條、形狀、寬度
    plt.plot(run, avg_pop, color='b', label='Avg fitness', linestyle='--', marker='o')
    plt.grid(axis='y')    #定義網格
    plt.legend(loc='lower right')   #當有兩種以上資料,可以使用.legend()來顯示資料的圖例

In [17]:
def train( training_data , periods , init_money):
    run = []
    best_fitness = []
    avg_fitness = []

    POP_SIZE = 400    #種群大小
    # DNA_LEN = 23    #雙均線策略 染色體長度 雙均線/各 8 bit (1~256) 投資比例 7 bit (0~100)
    DNA_LEN = 29    #布林通道策略 染色體長度 均線 8 bit (1~256) 投資比例 7 bit (0~100) # up_K 7 bit (0~127) down_K 7 bit (0 ~ 127)
    cross_rate = 0.7    #交配概率
    mutation_rate = 0.005   #變異概率

    #初始種群
    pop = init_population(POP_SIZE,DNA_LEN)


    #迭代次數
    generation_num = 30
    best_chromosome = ""
    best_fit = -1e9


    for i in range(generation_num):
        print(f"[Generation : {i+1}]")
        max_fit , avg_fit, pop = evaluate_fitness(pop , periods ,training_data , init_money) #計算適應度
        print(f" Best chromosome:{TwoMA_decode(sorted(pop, reverse=True, key = itemgetter(1))[0][0])}\n Best fitness:{max_fit}\n Avg fitness:{avg_fit}\n")
        # print(f" Best chromosome:{BollingerBand_decode(sorted(pop, reverse=True, key = itemgetter(1))[0][0])}\n Best fitness:{max_fit}\n Avg fitness:{avg_fit}\n")

        if best_fit < max_fit :
            best_fit = max_fit
            best_chromosome = sorted(pop, reverse=True, key = itemgetter(1))[0][0]

        pop = cross(pop,POP_SIZE,DNA_LEN,cross_rate)  #交配 
        pop = mutation(pop,DNA_LEN,mutation_rate) #變異

        best_fitness.append(max_fit)
        avg_fitness.append(avg_fit)
        run.append(i+1)

    draw_fitness(run,best_fitness,avg_fitness)

    return best_chromosome


In [20]:
if __name__ == '__main__':  
    data = pd.read_csv('2330.TW.csv')  #讀取csv資料檔



    init_year = 2012
    end_year = 2021

    split_year_li = [2014]

    # for i in range(len(data)):
    #     data['Date'][i] = data['Date'][i].replace('/','-')

    # for split_year in split_year_li:
    for split_year in range(init_year, end_year):

        train_data = pd.DataFrame(columns = list(data.columns.values))
        validation_data = pd.DataFrame(columns = list(data.columns.values))


        # 分割TV
        for index , row in data.iterrows():
            # print(type(row))
            
            if int(row["Date"].split("-")[0]) <= split_year:
                train_data = pd.concat([train_data , row.to_frame().T] )
                # print(train_data)
                # break
            else:
                validation_data = pd.concat([validation_data , row.to_frame().T])

        train_data = train_data.reset_index(drop = True)
        validation_data = validation_data.reset_index(drop = True)
        

        
        # train出最好的染色體
        best_chromosome = train(train_data , split_year - init_year + 1 , 10000 )
        TwoMA_validate(best_chromosome , validation_data , end_year - split_year , 10000 ) 
        # BollingerBand_validate(best_chromosome , validation_data , end_year - split_year , 10000 ) 

        # 最好的直接跑validation
        # TwoMA(validation_data , end_year - split_year , 92 , 219 , 10000 , 1.0 , print_graph = True)
        # BollingerBand(validation_data , end_year - split_year , 2 , 2.0625 , 0.0 , 10000 , 0.4960629921259843 , print_graph = True)
        # break
